# Movies Database Cleaning

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
original_movies_db = pd.read_excel('movies.xls')
original_movies_db.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22 00:00:00,89.0,8.1,Wil Wheaton,299174.0,Stephen King,1986.0
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11 00:00:00,103.0,7.8,Matthew Broderick,264740.0,John Hughes,1986.0
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16 00:00:00,110.0,6.9,Tom Cruise,236909.0,Jim Cash,1986.0
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18 00:00:00,137.0,8.4,Sigourney Weaver,540152.0,James Cameron,1986.0
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01 00:00:00,90.0,6.9,Joey Cramer,36636.0,Mark H. Baker,1986.0


In [3]:
movies_db = original_movies_db

In [4]:
# Filtering to just get 'USA' matches
#movies_db = movies_db[(movies_db['country'] == 'USA')]


# Filtering to just get 'BUDGET' not equal to 0.0
movies_db = movies_db[(movies_db['budget'] != 0.0)]


movies_db.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22 00:00:00,89.0,8.1,Wil Wheaton,299174.0,Stephen King,1986.0
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11 00:00:00,103.0,7.8,Matthew Broderick,264740.0,John Hughes,1986.0
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16 00:00:00,110.0,6.9,Tom Cruise,236909.0,Jim Cash,1986.0
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18 00:00:00,137.0,8.4,Sigourney Weaver,540152.0,James Cameron,1986.0
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01 00:00:00,90.0,6.9,Joey Cramer,36636.0,Mark H. Baker,1986.0


In [5]:
movies_db.loc[(movies_db['year'].isnull())]

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
199,18000000.0,J&M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1454,5000000.0,A&M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1714,12000000.0,A&M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4831,6000000.0,B&W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Exclude year nan rows

movies_db = movies_db.dropna()
movies_db.isnull().sum()

budget      0
company     0
country     0
director    0
genre       0
gross       0
name        0
rating      0
released    0
runtime     0
score       0
star        0
votes       0
writer      0
year        0
dtype: int64

In [7]:
# Checking column names

movies_db.columns

Index(['budget', 'company', 'country', 'director', 'genre', 'gross', 'name',
       'rating', 'released', 'runtime', 'score', 'star', 'votes', 'writer',
       'year'],
      dtype='object')

In [8]:
# Selecting the columns we want to keep
movies_db = movies_db[['year', 'name', 'budget', 'gross', 'score']]


# Renaming the columns
movies_db.columns = ['YEAR', 'TITLE', 'BUDGET', 'REVENUE', 'IMDB SCORE']

movies_db

,YEAR,TITLE,BUDGET,REVENUE,IMDB SCORE
0,1986.0,Stand by Me,8000000.0,52287414.0,8.1
1,1986.0,Ferris Bueller's Day Off,6000000.0,70136369.0,7.8
2,1986.0,Top Gun,15000000.0,179800601.0,6.9
3,1986.0,Aliens,18500000.0,85160248.0,8.4
4,1986.0,Flight of the Navigator,9000000.0,18564613.0,6.9
...,...,...,...,...,...
6808,2016.0,Love & Friendship,3000000.0,14013564.0,6.5
6809,2016.0,The Hollars,3800000.0,1016872.0,6.5
6811,2016.0,Middle School: The Worst Years of My Life,8500000.0,19985196.0,6.1
6814,2016.0,Risen,20000000.0,36874745.0,6.3


In [9]:
# Creating the '% BUDGET/REVENUE' column

movies_db['% BUDGET/REVENUE'] = round((movies_db['BUDGET'] / movies_db['REVENUE'] * 100), 1)


In [10]:
# Converting 'YEAR' column to 'int' type

movies_db['YEAR'] = movies_db['YEAR'].astype('str')
movies_db['YEAR'] = movies_db['YEAR'].apply(lambda x: x.split('.')[0])
movies_db['YEAR'] = movies_db['YEAR'].astype('int')

In [11]:
# Changing these 2 columns to be in millions
movies_db['BUDGET'] = round((movies_db['BUDGET'] / 1000000), 1)
movies_db['REVENUE'] = round((movies_db['REVENUE'] / 1000000), 1)

In [12]:
# Renaming columns to be in millions
movies_db.columns = ['YEAR', 'TITLE', 'BUDGET IN MILLIONS', 'REVENUE IN MILLIONS', 'IMDB SCORE', '% BUDGET/REVENUE']
movies_db.head()

,YEAR,TITLE,BUDGET IN MILLIONS,REVENUE IN MILLIONS,IMDB SCORE,% BUDGET/REVENUE
0,1986,Stand by Me,8.0,52.3,8.1,15.3
1,1986,Ferris Bueller's Day Off,6.0,70.1,7.8,8.6
2,1986,Top Gun,15.0,179.8,6.9,8.3
3,1986,Aliens,18.5,85.2,8.4,21.7
4,1986,Flight of the Navigator,9.0,18.6,6.9,48.5


In [13]:
# Final order
movies_db = movies_db[['YEAR', 'TITLE', 'BUDGET IN MILLIONS', 'REVENUE IN MILLIONS', '% BUDGET/REVENUE', 'IMDB SCORE']]

In [23]:
# Clean titles
movies_db['TITLE'] = movies_db['TITLE'].str.replace('-',' ').str.replace(',',' ').str.replace('.',' ').str.replace(':',' ').str.strip().str.title()
movies_db

,YEAR,TITLE,BUDGET IN MILLIONS,REVENUE IN MILLIONS,% BUDGET/REVENUE,IMDB SCORE
0,1986,Stand By Me,8.0,52.3,15.3,8.1
1,1986,Ferris Bueller'S Day Off,6.0,70.1,8.6,7.8
2,1986,Top Gun,15.0,179.8,8.3,6.9
3,1986,Aliens,18.5,85.2,21.7,8.4
4,1986,Flight Of The Navigator,9.0,18.6,48.5,6.9
...,...,...,...,...,...,...
6808,2016,Love & Friendship,3.0,14.0,21.4,6.5
6809,2016,The Hollars,3.8,1.0,373.7,6.5
6811,2016,Middle School The Worst Years Of My Life,8.5,20.0,42.5,6.1
6814,2016,Risen,20.0,36.9,54.2,6.3


### Awards winning movies

In [15]:
url = 'https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films'
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

# tag: tbody

table_body = soup.find_all('tbody')[0]
rows = [element.text for element in table_body.find_all('tr')]

clean_rows = [row.title().strip().replace('\n','  ').split('  ') for row in rows]

colnames = ['TITLE', 'YEAR', 'AWARDS', 'NOMINATIONS']
awards = pd.DataFrame(clean_rows[1:], columns = colnames)
awards.head()

,TITLE,YEAR,AWARDS,NOMINATIONS
0,Green Book,2018,3,5
1,Bohemian Rhapsody,2018,4,5
2,Roma,2018,3,10
3,Black Panther,2018,3,7
4,The Favourite,2018,1,10


In [16]:
# Like this it matches the movies' file year range
awards = awards[(awards['YEAR'] <= '2016') & (awards['YEAR'] >= '1986')]
awards['YEAR'] = awards['YEAR'].astype('int')

In [17]:
# Filter awards by len

awards['AWARDS'] = awards['AWARDS'].apply(lambda x: x.split(' ')[0])

In [18]:
# Resetting the index, as it was starting at 31
awards.reset_index()
awards_db = awards[['YEAR','TITLE', 'AWARDS', 'NOMINATIONS']]
awards_db.dtypes

YEAR            int64
TITLE          object
AWARDS         object
NOMINATIONS    object
dtype: object

In [19]:
all_db = pd.merge(movies_db, awards_db, on = 'TITLE', how = 'outer')

In [20]:
all_db = all_db.sort_values(['TITLE'])
all_db.reset_index()

#all_db[(all_db['AWARDS'] == '1')].head(50)
all_db.isnull().sum()

YEAR_x                  191
TITLE                     9
BUDGET IN MILLIONS      191
REVENUE IN MILLIONS     191
% BUDGET/REVENUE        191
IMDB SCORE              191
YEAR_y                 4374
AWARDS                 4374
NOMINATIONS            4374
dtype: int64